<a href="https://colab.research.google.com/github/Ananya-AJ/Deep-Learning/blob/main/Assignment4/k_data_augmentation_classification_timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neural network to classify temperatures as high and low. simple time series dataset that you can use to train a neural network and test for accuracy. This dataset consists of daily minimum temperatures in Melbourne, Australia from 1981 to 1990.

In [2]:
!pip install tsaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tsaug import TimeWarp, Crop, Drift, AddNoise
import tsaug
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
dataset = pd.read_csv(url, header=0, index_col=0)

# Convert the dataset to a numpy array
data = dataset.values

# Split the dataset into training and testing sets
train_size = int(len(data) * 0.67)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]

# Function to create the time series dataset
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# Reshape the data into the appropriate format for the neural network
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

# Define the neural network model
model = keras.Sequential([
    keras.layers.LSTM(4, input_shape=(look_back, 1)),
    keras.layers.Dense(1)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)

# Evaluate the model on the testing set
scores = model.evaluate(testX, testY, verbose=0)
print("Test loss:", scores)



Define a function to perform data augmentation on the training data. This function will randomly apply a set of transformations to the data, including timewrap, drift and reverse

In [6]:
from tsaug import TimeWarp, Drift, Reverse

# Define the augmentations
augmenter = (
    TimeWarp() 
    + Drift(max_drift=(0.1, 0.3)) 
    + Reverse()
)

# Apply the augmentations to the training data
trainX_aug = np.zeros_like(trainX)
for i in range(len(trainX)):
    trainX_aug[i, :, 0] = augmenter.augment(trainX[i, :, 0])
    
# Apply the augmentations to the testing data
testX_aug = np.zeros_like(testX)
for i in range(len(testX)):
    testX_aug[i, :, 0] = augmenter.augment(testX[i, :, 0])

Train the model using data augmentation. We will run for 20 epochs, test the model

In [ ]:
# Train the model on the augmented training data
model.fit(trainX_aug, trainY, epochs=20, batch_size=1, verbose=2)

# Evaluate the model on the augmented testing data
scores_aug = model.evaluate(testX_aug, testY, verbose=0)
print("Test loss (augmented data):", scores_aug)
